In [1]:
from CoolProp.CoolProp import PropsSI
import random 
# use the letters:
# ... T for (T)emperature 
# ... P for (P)ressure
# ... D for (D)ensity (use the density to calculate the specific volume)
# ... Q for vapor quality x

In [2]:
#Boundary conditions

#Ambiant air
p0 = 1.03e5 #Pa        TBD
T0 = 273+12 #K         TBD

#Miscellaneous
P = 1e6 #W             TBD
R = 8.3145 #J / mol K  
Mm = 28.97e-3 #kg/mol
cv = 718 #J/ kg K      TBD
dt =1  #s
p_loss=0.001 #bar       TBD

#Cavern
V = 500  #m^3          TBD
pmax = 80e5 #bar       TBD
m0 = p0*V/T0/R*Mm #kg

#Compressor
eff_c   = 0.9 #[-]
p_ratio_c = 2 #[-]

#Turbine
eff_t   = 0.9 #[-]
p_ratio_t = 2 #[-]

#Requested power
Pout=10*P #W           TBD


In [3]:
#Functions

#Get temperature after (de)pressurising from original temp
def Tpres(T,p_ratio):
    return T*p_ratio**(R/(Mm*cv+R))

#Get enthalpy for known T,p
def hTp(p,T):
    return PropsSI("H","P",p,"T",T,"air")

#First approximation specific work using enthalpy
def W1(p_in,p_ratio,T_in):
    p_out = p_in*p_ratio       
    T_out = Tpres(T_in,p_ratio)
    h_in = hTp(p_in,T_in)
    h_out = hTp(p_in,T_out)
    return h_out-h_in

#Mass flow rate calculation based on input power, efficiencies and required specific work
def mdot1(p_ratio):
    Work1 = W1(p0,p_ratio,T0)
    Work2 = W1(p0*p_ratio,p_ratio,T0)
    m=P*eff_c/(Work1+Work2)
    return m

#Second approximation specific work including velocities
def W2():
    #...
    return "null"

#Mass flow rate corrected for kinetic changes
def mdot2():
    #...
    return "null"

#Function to fill the cavern
def fill_cavern(tmax):
    
    #Required argument for python while loop
    global p_cavern
    global m
    global t
    
    #Fill cavern until full or no more power (tmax is max time there is power)
    while p_cavern < pmax and t < tmax:
        
        #Determine required pressure ratio
        p_ratio = (p_cavern/p0)**(1/2) + p_loss

        #Determine mass in cavern
        m += mdot1(p_ratio)*dt

        #Determine pressure in cavern
        p_cavern = m*R/Mm*T0/V
    
        t += dt
    if t < tmax:
        print('Not all this power can be stored, {0:.1f}% can not be used!'.format((1-t/tmax)*100))
        print("It took {0:.2f} hours to fill the cavern.".format(t/3600))
    else:
        print("No more power available.")
        
    print("Now the pressure is {0:.2f} bar".format(p_cavern/1e5))
    t=0




In [4]:
#Work in progress functions
def empty_cavern(tmax):
    global p_cavern
    global m
    global t
    
    while p_cavern > 2*p0 and t < tmax:
        p_ratio = (p0/p_cavern)**(1/2)
        m += Pout/eff_t/(W1(p_cavern,p_ratio,T0)+W1(p_cavern*p_ratio,p_ratio,T0))
        p_cavern = m*R/Mm*T0/V
    
        t += dt
    
    if t < tmax:
        print("No more power available, ran out at {0:.2f} hours.".format(t/3600))
    else:
        print("Not out of energy :).")
        
    print("Now the pressure is {0:.2f} bar".format(p_cavern/1e5))
    t=0




    


In [5]:
#Reset conditions
p_cavern = p0
m = m0
t = 0

In [6]:
#First fill for x hours
fill_cavern(3600*4)

No more power available.
Now the pressure is 66.14 bar


In [7]:
#Second fill for y hour(s)
fill_cavern(3600*1.5)

Not all this power can be stored, 15.1% can not be used!
It took 1.27 hours to fill the cavern.
Now the pressure is 80.00 bar


In [8]:
#Extracting energy from cavern
empty_cavern(3600*0.1)

Not out of energy :).
Now the pressure is 59.81 bar


In [9]:
#Extracting energy from cavern
empty_cavern(3600*0.3)

No more power available, ran out at 0.20 hours.
Now the pressure is 1.91 bar
